# haystack with docling

In [1]:
%pip install -q --progress-bar off --no-warn-conflicts docling-haystack haystack-ai docling pymilvus milvus-haystack sentence-transformers python-dotenv

In [2]:
import os
from pathlib import Path
from tempfile import mkdtemp

from docling_haystack.converter import ExportType
from dotenv import load_dotenv

def _get_env_from_colab_or_os(key):
    try:
        from google.colab import userdata

        try:
            return userdata.get(key)
        except userdata.SecretNotFoundError:
            pass
    except ImportError:
        pass
    return os.getenv(key)

load_dotenv()
HF_TOKEN = _get_env_from_colab_or_os("HF_TOKEN")
PATHS = ["/content/Codex_over_het_welzijn_op_het_werk.pdf"]  # Docling Technical Report
EMBED_MODEL_ID = "sentence-transformers/all-MiniLM-L6-v2"
GENERATION_MODEL_ID = "mistralai/Mixtral-8x7B-Instruct-v0.1"
EXPORT_TYPE = ExportType.DOC_CHUNKS
TOP_K = 3
MILVUS_URI = str("/content/docling.db")
QUESTION = "Wat is de minimale temperatuur van de lucht voor werknemers?"

Indexing pipeline
takes 60mins on CPU for 700 page doc

In [3]:
from docling_haystack.converter import DoclingConverter
from haystack import Pipeline
from haystack.components.embedders import (
    SentenceTransformersDocumentEmbedder,
    SentenceTransformersTextEmbedder,
)
from haystack.components.preprocessors import DocumentSplitter
from haystack.components.writers import DocumentWriter
from milvus_haystack import MilvusDocumentStore, MilvusEmbeddingRetriever

from docling.chunking import HybridChunker

document_store = MilvusDocumentStore(
    connection_args={"uri": MILVUS_URI},
    drop_old=True,
    text_field="txt",  # set for preventing conflict with same-name metadata field
)

idx_pipe = Pipeline()
idx_pipe.add_component(
    "converter",
    DoclingConverter(
        export_type=EXPORT_TYPE,
        chunker=HybridChunker(tokenizer=EMBED_MODEL_ID),
    ),
)
idx_pipe.add_component(
    "embedder",
    SentenceTransformersDocumentEmbedder(model=EMBED_MODEL_ID),
)
idx_pipe.add_component("writer", DocumentWriter(document_store=document_store))
if EXPORT_TYPE == ExportType.DOC_CHUNKS:
    idx_pipe.connect("converter", "embedder")
elif EXPORT_TYPE == ExportType.MARKDOWN:
    idx_pipe.add_component(
        "splitter",
        DocumentSplitter(split_by="sentence", split_length=1),
    )
    idx_pipe.connect("converter.documents", "splitter.documents")
    idx_pipe.connect("splitter.documents", "embedder.documents")
else:
    raise ValueError(f"Unexpected export type: {EXPORT_TYPE}")
idx_pipe.connect("embedder", "writer")
idx_pipe.run({"converter": {"paths": PATHS}})

DEBUG:pymilvus.milvus_client.milvus_client:Created new connection using: 60fa43a9a1dc4be283e4c2c07e1485a3


tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/90.9M [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (1135 > 512). Running this sequence through the model will result in indexing errors


Batches:   0%|          | 0/66 [00:00<?, ?it/s]

{'writer': {'documents_written': 2104}}

RAG pipeline¶


In [8]:
from haystack.components.builders import AnswerBuilder
from haystack.components.builders.prompt_builder import PromptBuilder
from haystack.components.generators import HuggingFaceAPIGenerator
from haystack.utils import Secret

prompt_template = """
    Beantwoord de vraag met behulp van volgende documenten.
    Documenten:
    {% for doc in documents %}
        {{ doc.content }}
    {% endfor %}
    Vraag: {{query}}
    Antwoord:
    """

rag_pipe = Pipeline()
rag_pipe.add_component(
    "embedder",
    SentenceTransformersTextEmbedder(model=EMBED_MODEL_ID),
)
rag_pipe.add_component(
    "retriever",
    MilvusEmbeddingRetriever(document_store=document_store, top_k=TOP_K),
)
rag_pipe.add_component("prompt_builder", PromptBuilder(template=prompt_template))
rag_pipe.add_component(
    "llm",
    HuggingFaceAPIGenerator(
        api_type="serverless_inference_api",
        api_params={"model": GENERATION_MODEL_ID},
        token=Secret.from_token(HF_TOKEN) if HF_TOKEN else None,
    ),
)
rag_pipe.add_component("answer_builder", AnswerBuilder())
rag_pipe.connect("embedder.embedding", "retriever")
rag_pipe.connect("retriever", "prompt_builder.documents")
rag_pipe.connect("prompt_builder", "llm")
rag_pipe.connect("llm.replies", "answer_builder.replies")
rag_pipe.connect("llm.meta", "answer_builder.meta")
rag_pipe.connect("retriever", "answer_builder.documents")


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

query

In [14]:
QUESTION = '''Wanneer moet je als bedrijf verplicht een interne preventieadviseur hebben?
A: Dat is altijd verplicht
B: dat is nooit verplicht, enkel aangeraden
C: Verplicht vanaf 20 of meer werknemers'''

In [17]:
QUESTION = '''Wat is geen taak van de Interne Dienst voor Preventie en Bescherming op het Werk (IDPBW)?
A: Risico's onderzoeken en advies geven over de risico-evaluatie
B: Advies geven over arbeidshygiëne
C: Werk-privébalans voor werknemers bewaken
D: Policies rond telewerken goedkeuren
E: Werkinstructies opstellen'''

In [20]:
QUESTION = '''Moeten al je werknemers een medisch onderzoek of medisch toezicht ondergaan?
A: Ja, iedereen moet een medisch onderzoek ondergaan
B: Nee, het is alleen verplicht voor oudere medewerkers
C: Nee, het is niet verplicht voor alle werknemers
'''


In [23]:
QUESTION = '''Ik organiseer een griepvaccinatiecampagne in mijn bedrijf. Zijn mijn werknemers verplicht om zich te laten vaccineren?
A: Ja, dat is verplicht
B: Nee, dat is niet verplicht
'''

In [26]:
QUESTION = '''Ik heb een werknemer die sinds 7 maanden arbeidsongeschikt is. Kan ik een aanvraag tot vaststelling van definitieve ongeschiktheid indienen?
A: Ja
B: Nee, ik moet wachten tot er een ononderbroken periode van minstens 9 maanden arbeidsongeschiktheid is
'''

In [29]:
QUESTION = '''Moet je een externe preventieadviseur toegang geven tot je bedrijf om de werkplek te bezoeken?
A: Ja, dat is verplicht
B: Nee, dat is niet verplicht
'''

In [32]:
QUESTION = '''Je bedrijf heeft 55 mensen in dienst. Ben je verplicht om een vertrouwenspersoon aan te stellen?
A: Ja, dat is verplicht
B: Nee, dat is niet verplicht
'''

In [33]:
rag_res = rag_pipe.run(
    {
        "embedder": {"text": QUESTION},
        "prompt_builder": {"query": QUESTION},
        "answer_builder": {"query": QUESTION},
    }
)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [34]:
from docling.chunking import DocChunk


print(f"Question:\n{QUESTION}\n")
print(f"Answer:\n{rag_res['answer_builder']['answers'][0].data.strip()}\n")
print("Sources:")
sources = rag_res["answer_builder"]["answers"][0].documents
for source in sources:
    if EXPORT_TYPE == ExportType.DOC_CHUNKS:
        doc_chunk = DocChunk.model_validate(source.meta["dl_meta"])
        print(f"- text: {repr(doc_chunk.text)}")
        if doc_chunk.meta.origin:
            print(f"  file: {doc_chunk.meta.origin.filename}")
        if doc_chunk.meta.headings:
            print(f"  section: {' / '.join(doc_chunk.meta.headings)}")
        bbox = doc_chunk.meta.doc_items[0].prov[0].bbox
        print(
            f"  page: {doc_chunk.meta.doc_items[0].prov[0].page_no}, "
            f"bounding box: [{int(bbox.l)}, {int(bbox.t)}, {int(bbox.r)}, {int(bbox.b)}]"
        )
    elif EXPORT_TYPE == ExportType.MARKDOWN:
        print(repr(source.content))
    else:
        raise ValueError(f"Unexpected export type: {EXPORT_TYPE}")

Question:
Je bedrijf heeft 55 mensen in dienst. Ben je verplicht om een vertrouwenspersoon aan te stellen?
A: Ja, dat is verplicht
B: Nee, dat is niet verplicht


Answer:
B: Nee, dat is niet verplicht.
    De aanstelling van een vertrouwenspersoon is niet verplicht voor bedrijven met minder dan 50 werknemers.
    (Zie opmerking vooraf in het lastenboek, waarin staat dat de maatregelen in beginsel moeten worden toegepast, tenzij uit de resultaten van de beoordeling anders blijkt.)
    Deze beoordeling is niet beschreven in het lastenboek, dus wordt aangenomen dat de maatregelen in beginsel niet van toepassing zijn op bedrijven met minder dan 50 werknemers.
    (Zie ook artikel I.6-10, eerste lid, waarin staat dat het verslag moet worden ingevuld op basis van de methodiek in het lastenboek.)

Sources:
- text: 'Voor het invullen van het verslag wordt rekening gehouden met de volgende elementen en verwijzingen:'
  file: Codex_over_het_welzijn_op_het_werk.pdf
  section: Lastenboek houdende 

Quiz: Ken je jouw verplichtingen rond welzijn op het werk?

Je scoorde tussen 3 en 5:
Je bent duidelijk op de goede weg. Je kent als werkgever al de meeste van je verplichtingen omtrent welzijn op het werk. Maar er zijn nog een paar zaken die verduidelijking nodig hebben. Reken op onze Externe Dienst voor Preventie en Bescherming op het Werk (EDPBW) om al je vragen te beantwoorden.

1. Wanneer moet je als bedrijf verplicht een interne preventieadviseur hebben?
Je antwoordde fout.

Het correcte antwoord is: Dat is altijd verplicht.

Als werkgever heb je de wettelijke verantwoordelijk om het welzijn van je werknemers op het werk te bevorderen. Daarom ben je verplicht om een Interne Dienst voor Preventie en Bescherming op het Werk (IDPBW) op te richten. Binnen die dienst worden dan één of meerdere werknemers als preventieadviseur aangesteld, afhankelijk van de grootte van het bedrijf.

Als je bedrijf minder dan 20 werknemers tewerkstelt, mag je als werkgever zelf de rol van preventieadviseur opnemen.

2. Wat is geen taak van de Interne Dienst voor Preventie en Bescherming op het Werk (IDPBW)?  
Je antwoordde fout.  

Het correcte antwoord is: Policies rond telewerken goedkeuren.

De IDPBW staat de werkgever en de werknemer bij om de regelgeving inzake welzijn, de veiligheid en gezondheid op de werkvloer in de praktijk toe te passen.

Die taken omvatten:

Arbeidsveiligheid, zoals het onderzoeken en beperken van risico’s en (veilige) werkinstructies opstellen
Arbeidshygiëne en arbeidsgeneeskunde
Ergonomie
Psychosociale aspecten, zoals het helpen bewaken van de werk-privébalans
De IDPBW geeft advies aan de bedrijfsleiding, formuleert richtlijnen voor een beleid, maakt risicoanalyses en stelt zelfs actieplannen op. Het uitvoeren van het advies is echter de verantwoordelijkheid van de werkgever.

3. Moeten al je werknemers een medische onderzoek of medisch toezicht ondergaan?
Je antwoordde correct.

Slechts enkele profielen moeten verplicht jaarlijks onderworpen worden aan een gezondheidstoezicht.

De risicoanalyse van werkplekken bepaalt of de functie wel of niet onder medisch toezicht valt.   

Periodiek gezondheidstoezicht is verplicht voor veiligheidswerkplekken, waakwerkplekken en werkzaamheden met een bepaald risico. De frequentie is afhankelijk van het risiconiveau.   

Dit toezicht bestaat uit twee delen:  

Een periodieke gezondheidsbeoordeling door de bedrijfsarts (anamnese en klinisch onderzoek)  
Aanvullende medische procedures: verpleegkundigen voeren tussentijds medisch toezicht uit in de vorm van aanvullende medische procedures (vaccinatie, longfunctietest, audiogram, elektrocardiogram, vragenlijst, ...) Afhankelijk van de resultaten van de onderzoeken, of op verzoek van de werknemer, kan altijd een afspraak met de bedrijfsarts worden overwogen
Beeldschermwerkplekken brengen geen risico's met zich mee waarvoor medisch toezicht nodig is.

4. Ik organiseer een griepvaccinatiecampagne in mijn bedrijf. Zijn mijn werknemers verplicht om zich te laten vaccineren?
Je antwoordde correct.

Nee, dat is niet verplicht.

Verplichte vaccinaties zijn duidelijk gedefinieerd in de wetgeving, de griepvaccinatie valt daar niet onder. Onder deze verplichte vaccinaties vallen de tetanusvaccinatie, de tuberculosevaccinatie (of Mantoux-test) en de hepatitis B-vaccinatie, maar alleen in specifieke sectoren waar werknemers worden blootgesteld aan het betrokken biologisch agens. Bijvoorbeeld, de tetanusvaccinatie is verplicht voor werknemers in de reinigingsdiensten van de openbare weg.

De werkgever kan de griepvaccinatie aan zijn personeel aanbieden. Het aanbieden van deze vaccinatie kan een positieve impact hebben op het ziekteverzuim.

Het wordt vooral aanbevolen in ziekenhuizen en zorginstellingen, omdat het bijdraagt aan het verminderen van de verspreiding van infecties in omgevingen waar kwetsbare personen aanwezig zijn.  

5. Ik heb een werknemer die sinds 7 maanden arbeidsongeschikt is. Kan ik een aanvraag tot vaststelling van definitieve ongeschiktheid indienen?
Je antwoordde correct.

Nee, ik moet wachten tot er een ononderbroken periode van minstens 9 maanden arbeidsongeschiktheid is.

Zowel de werknemer als de werkgever kunnen de specifieke procedure opstarten, op voorwaarde dat:

Er geen re-integratietraject aan de gang is in het bedrijf voor de werknemer die arbeidsongeschikt is.
De werknemer gedurende een ononderbroken periode van minstens 9 maanden arbeidsongeschikt is. Opmerking: een terugkeer naar het werk (volledig of gedeeltelijk) van minder dan 14 dagen onderbreekt de periode van arbeidsongeschiktheid niet.
Let op, er moet een specifieke procedure worden gevolgd om deze aanvraag in te dienen. De kennisgeving van de aanvraag moet per aangetekende post naar de andere partij (werkgever of werknemer) en naar de arbeidsgeneesheer worden gestuurd.

Als de werkgever de aanvraag indient, moet de kennisgeving aan de werknemer volgende vermelden:

Het recht van de werknemer om de preventieadviseur-arbeidsgeneesheer te vragen de mogelijkheden voor aangepast werk of andere opties te onderzoeken, indien vastgesteld wordt dat de werknemer het overeengekomen werk niet meer kan uitvoeren.
Het recht van de werknemer om tijdens deze procedure bijgestaan te worden door de syndicale delegatie van het bedrijf.
Als je aangesloten bent bij onze externe preventiedienst, nodigen we je uit om de templates van Securex te gebruiken.

6. Moet je een externe preventieadviseur toegang geven tot je bedrijf om de werkplek te bezoeken?
Je antwoordde correct.

Ja, dat is verplicht.

Het bedrijfsbezoek omvat een rondleiding op de werkplek, begeleid door de werkgever en een preventieadviseur van de externe dienst. De Externe Dienst voor Preventie en Bescherming op het werk (EDPBW) bezorgt de werkgever vervolgens een rapport waarin de bevindingen duidelijk en concreet worden uiteengezet. Het tijdsbestek waarbinnen dit bezoek moet worden georganiseerd, wordt beïnvloed door de tariefgroep waartoe het bedrijf behoort, die op zijn beurt wordt bepaald door de hoofdactiviteit van de werkgever.

7. Je bedrijf heeft 55 mensen in dienst. Ben je verplicht om een vertrouwenspersoon aan te stellen?   
Je antwoordde fout.

Het correcte antwoord is: Ja, dat is verplicht.

Een vertrouwenspersoon aanstellen is vanaf 1 december 2023 verplicht voor bedrijven met 50 werknemers of meer.

De vertrouwenspersoon kan een reële impact hebben op het welzijn van collega's. Bij conflicten, pesterijen, discriminatie, enz. kan de vertrouwenspersoon advies, begeleiding of ondersteuning bieden.

# Document search with embeddings

<table class="tfo-notebook-buttons" align="left">
      <td>
    <a target="_blank" href="https://ai.google.dev/gemini-api/tutorials/document_search"><img src="https://ai.google.dev/static/site-assets/images/docs/notebook-site-button.png" height="32" width="32" />View on ai.google.dev</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/google/generative-ai-docs/blob/main/site/en/gemini-api/tutorials/document_search.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/google/generative-ai-docs/blob/main/site/en/gemini-api/tutorials/document_search.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
</table>

## Overview

This example demonstrates how to use the Gemini API to create embeddings so that you can perform document search. You will use the Python client library to build a word embedding that allows you to compare search strings, or questions, to document contents.

In this tutorial, you'll use embeddings to perform document search over a set of documents to ask questions related to the Google Car.

## Prerequisites

You can run this quickstart in Google Colab.

To complete this quickstart on your own development environment, ensure that your environment meets the following requirements:

-  Python 3.9+
-  An installation of `jupyter` to run the notebook.

## Setup

First, download and install the Gemini API Python library.

In [ ]:
!pip install -U -q google-generativeai

In [ ]:
import textwrap
import numpy as np
import pandas as pd

import google.generativeai as genai

# Used to securely store your API key
from google.colab import userdata

from IPython.display import Markdown

### Grab an API Key

Before you can use the Gemini API, you must first obtain an API key. If you don't already have one, create a key with one click in Google AI Studio.

<a class="button button-primary" href="https://makersuite.google.com/app/apikey" target="_blank" rel="noopener noreferrer">Get an API key</a>

In Colab, add the key to the secrets manager under the "🔑" in the left panel. Give it the name `API_KEY`.

Once you have the API key, pass it to the SDK. You can do this in two ways:

* Put the key in the `GOOGLE_API_KEY` environment variable (the SDK will automatically pick it up from there).
* Pass the key to `genai.configure(api_key=...)`

In [ ]:
# Or use `os.getenv('API_KEY')` to fetch an environment variable.
API_KEY=userdata.get('API_KEY')

genai.configure(api_key=API_KEY)

Key Point: Next, you will choose a model. Any embedding model will work for this tutorial, but for real applications it's important to choose a specific model and stick with it. The outputs of different models are not compatible with each other.

**Note**: At this time, the Gemini API is [only available in certain regions](https://ai.google.dev/gemini-api/docs/available-regions).

In [ ]:
for m in genai.list_models():
  if 'embedContent' in m.supported_generation_methods:
    print(m.name)

models/embedding-001
models/embedding-001


## Embedding generation

In this section, you will see how to generate embeddings for a piece of text using the embeddings from the Gemini API.

### API changes to Embeddings with model embedding-001

For the new embeddings model, embedding-001, there is a new task type parameter and the optional title (only valid with task_type=`RETRIEVAL_DOCUMENT`).

These new parameters apply only to the newest embeddings models.The task types are:

Task Type | Description
---       | ---
RETRIEVAL_QUERY	| Specifies the given text is a query in a search/retrieval setting.
RETRIEVAL_DOCUMENT | Specifies the given text is a document in a search/retrieval setting.
SEMANTIC_SIMILARITY	| Specifies the given text will be used for Semantic Textual Similarity (STS).
CLASSIFICATION	| Specifies that the embeddings will be used for classification.
CLUSTERING	| Specifies that the embeddings will be used for clustering.

Note: Specifying a `title` for `RETRIEVAL_DOCUMENT` provides better quality embeddings for retrieval.

In [ ]:
title = "The next generation of AI for developers and Google Workspace"
sample_text = ("Title: The next generation of AI for developers and Google Workspace"
    "\n"
    "Full article:\n"
    "\n"
    "Gemini API & Google AI Studio: An approachable way to explore and prototype with generative AI applications")

model = 'models/embedding-001'
embedding = genai.embed_content(model=model,
                                content=sample_text,
                                task_type="retrieval_document",
                                title=title)

print(embedding)

{'embedding': [0.034585103, -0.044509504, -0.027291223, 0.0072681927, 0.061689284, 0.03362112, 0.028627988, 0.022681564, 0.04958079, 0.07274552, 0.011150464, 0.04200501, -0.029782884, -0.0041767005, 0.05074771, -0.056339227, 0.051204756, 0.04734613, -0.022025354, 0.025162602, 0.046016376, -0.003416976, -0.024010269, -0.044340927, -0.01520864, -0.013577372, -0.009918958, -0.028144406, -0.00024770075, 0.031201784, -0.072506696, 0.022366496, -0.032672316, -0.0025522006, -0.0019957912, -0.023193765, -0.020633291, -0.014031609, -0.00071676675, -0.0073200124, 0.014770645, -0.09390713, -0.017846372, 0.032825496, 0.017616265, -0.046674345, 0.03469292, 0.03386835, 0.0028274113, -0.07737739, 0.023789782, 0.025950644, 0.06952142, -0.029875675, -0.018693604, 0.007266584, -0.0067282487, 0.000802912, 0.020609016, 0.012406181, -0.018825717, 0.051171597, -0.0080359895, 0.008457639, 0.01197146, -0.080320396, -0.040698495, 0.0018266322, 0.042915005, 0.021464704, 0.022519842, 0.0059912056, 0.050887667, -

## Building an embeddings database

Here are three sample texts to use to build the embeddings database. You will use the Gemini API to create embeddings of each of the documents. Turn them into a dataframe for better visualization.

In [ ]:
DOCUMENT1 = {
    "title": "Operating the Climate Control System",
    "content": "Your Googlecar has a climate control system that allows you to adjust the temperature and airflow in the car. To operate the climate control system, use the buttons and knobs located on the center console.  Temperature: The temperature knob controls the temperature inside the car. Turn the knob clockwise to increase the temperature or counterclockwise to decrease the temperature. Airflow: The airflow knob controls the amount of airflow inside the car. Turn the knob clockwise to increase the airflow or counterclockwise to decrease the airflow. Fan speed: The fan speed knob controls the speed of the fan. Turn the knob clockwise to increase the fan speed or counterclockwise to decrease the fan speed. Mode: The mode button allows you to select the desired mode. The available modes are: Auto: The car will automatically adjust the temperature and airflow to maintain a comfortable level. Cool: The car will blow cool air into the car. Heat: The car will blow warm air into the car. Defrost: The car will blow warm air onto the windshield to defrost it."}
DOCUMENT2 = {
    "title": "Touchscreen",
    "content": "Your Googlecar has a large touchscreen display that provides access to a variety of features, including navigation, entertainment, and climate control. To use the touchscreen display, simply touch the desired icon.  For example, you can touch the \"Navigation\" icon to get directions to your destination or touch the \"Music\" icon to play your favorite songs."}
DOCUMENT3 = {
    "title": "Shifting Gears",
    "content": "Your Googlecar has an automatic transmission. To shift gears, simply move the shift lever to the desired position.  Park: This position is used when you are parked. The wheels are locked and the car cannot move. Reverse: This position is used to back up. Neutral: This position is used when you are stopped at a light or in traffic. The car is not in gear and will not move unless you press the gas pedal. Drive: This position is used to drive forward. Low: This position is used for driving in snow or other slippery conditions."}

documents = [DOCUMENT1, DOCUMENT2, DOCUMENT3]

Organize the contents of the dictionary into a dataframe for better visualization.

In [ ]:
df = pd.DataFrame(documents)
df.columns = ['Title', 'Text']
df

,Text
0,Operating the Climate Control System Your Goo...
1,Your Googlecar has a large touchscreen display...
2,Shifting Gears Your Googlecar has an automati...


Get the embeddings for each of these bodies of text. Add this information to the dataframe.

In [ ]:
# Get the embeddings of each text and add to an embeddings column in the dataframe
def embed_fn(title, text):
  return genai.embed_content(model=model,
                             content=text,
                             task_type="retrieval_document",
                             title=title)["embedding"]

df['Embeddings'] = df.apply(lambda row: embed_fn(row['Title'], row['Text']), axis=1)
df

,Text,Embeddings
0,Operating the Climate Control System Your Goo...,"[-0.016329883, -0.01747576, -0.038270127, -0.0..."
1,Your Googlecar has a large touchscreen display...,"[0.024793636, -0.024769256, -0.01176664, -0.04..."
2,Shifting Gears Your Googlecar has an automati...,"[-0.025426013, 0.00023183432, -0.02406427, -0...."


## Document search with Q&A

Now that the embeddings are generated, let's create a Q&A system to search these documents. You will ask a question about hyperparameter tuning, create an embedding of the question, and compare it against the collection of embeddings in the dataframe.

The embedding of the question will be a vector (list of float values), which will be compared against the vector of the documents using the dot product. This vector returned from the API is already normalized. The dot product represents the similarity in direction between two vectors.

The values of the dot product can range between -1 and 1, inclusive. If the dot product between two vectors is 1, then the vectors are in the same direction. If the dot product value is 0, then these vectors are orthogonal, or unrelated, to each other. Lastly, if the dot product is -1, then the vectors point in the opposite direction and are not similar to each other.

Note, with the new embeddings model (`embedding-001`), specify the task type as `QUERY` for user query and `DOCUMENT` when embedding a document text.

Task Type | Description
---       | ---
RETRIEVAL_QUERY	| Specifies the given text is a query in a search/retrieval setting.
RETRIEVAL_DOCUMENT | Specifies the given text is a document in a search/retrieval setting.

In [ ]:
query = "How do you shift gears in the Google car?"
model = 'models/embedding-001'

request = genai.embed_content(model=model,
                              content=query,
                              task_type="retrieval_query")

Use the `find_best_passage` function to calculate the dot products, and then sort the dataframe from the largest to smallest dot product value to retrieve the relevant passage out of the database.

In [ ]:
def find_best_passage(query, dataframe):
  """
  Compute the distances between the query and each document in the dataframe
  using the dot product.
  """
  query_embedding = genai.embed_content(model=model,
                                        content=query,
                                        task_type="retrieval_query")
  dot_products = np.dot(np.stack(dataframe['Embeddings']), query_embedding["embedding"])
  idx = np.argmax(dot_products)
  return dataframe.iloc[idx]['Text'] # Return text from index with max value

View the most relevant document from the database:

In [ ]:
passage = find_best_passage(query, df)
passage

'Shifting Gears  Your Googlecar has an automatic transmission. To shift gears, simply move the shift lever to the desired position.  Park: This position is used when you are parked. The wheels are locked and the car cannot move. Reverse: This position is used to back up. Neutral: This position is used when you are stopped at a light or in traffic. The car is not in gear and will not move unless you press the gas pedal. Drive: This position is used to drive forward. Low: This position is used for driving in snow or other slippery conditions.'

## Question and Answering Application

Let's try to use the text generation API to create a Q & A system. Input your own custom data below to create a simple question and answering example. You will still use the dot product as a metric of similarity.

In [ ]:
def make_prompt(query, relevant_passage):
  escaped = relevant_passage.replace("'", "").replace('"', "").replace("\n", " ")
  prompt = textwrap.dedent("""You are a helpful and informative bot that answers questions using text from the reference passage included below. \
  Be sure to respond in a complete sentence, being comprehensive, including all relevant background information. \
  However, you are talking to a non-technical audience, so be sure to break down complicated concepts and \
  strike a friendly and converstional tone. \
  If the passage is irrelevant to the answer, you may ignore it.
  QUESTION: '{query}'
  PASSAGE: '{relevant_passage}'

    ANSWER:
  """).format(query=query, relevant_passage=escaped)

  return prompt

In [ ]:
prompt = make_prompt(query, passage)
print(prompt)

You are a helpful and informative bot that answers questions using text from the reference passage included below.   Be sure to respond in a complete sentence, being comprehensive, including all relevant background information.   However, you are talking to a non-technical audience, so be sure to break down complicated concepts and   strike a friendly and converstional tone.   If the passage is irrelevant to the answer, you may ignore it.
  QUESTION: 'How do you shift gears in the Google car?'
  PASSAGE: 'Shifting Gears  Your Googlecar has an automatic transmission. To shift gears, simply move the shift lever to the desired position.  Park: This position is used when you are parked. The wheels are locked and the car cannot move. Reverse: This position is used to back up. Neutral: This position is used when you are stopped at a light or in traffic. The car is not in gear and will not move unless you press the gas pedal. Drive: This position is used to drive forward. Low: This position i

Choose one of the Gemini content generation models in order to find the answer to your query.

In [ ]:
for m in genai.list_models():
  if 'generateContent' in m.supported_generation_methods:
    print(m.name)

models/gemini-1.5-pro
models/gemini-1.5-flash


In [ ]:
model = genai.GenerativeModel('gemini-1.5-pro-latest')
answer = model.generate_content(prompt)

In [ ]:
Markdown(answer.text)

The provided passage does not contain information about how to shift gears in a Google car, so I cannot answer your question from this source.

## Next steps

To learn how to use other services in the Gemini API, see the [Python quickstart](https://ai.google.dev/tutorials/python_quickstart).

To learn more about how you can use embeddings, see these  other tutorials:

 * [Anomaly Detection with Embeddings](https://ai.google.dev/gemini-api/tutorials/anomaly_detection)
 * [Clustering with Embeddings](https://ai.google.dev/gemini-api/tutorials/clustering_with_embeddings)
 * [Training a Text Classifier with Embeddings](https://ai.google.dev/gemini-api/tutorials/text_classifier_embeddings)